In [37]:
from datasets import load_dataset
import pandas as pd
import re
import html
import contractions
import emoji
import nltk
from nltk.corpus import stopwords
from huggingface_hub import list_datasets

In [38]:
emoticon_map = {
    ":)": "😊",
    "=)": "😊",
    ":]": "😊",
    ":(": "😞",
    "=(": "😞",
    ";)": "😉",
    ":D": "😁",
    ":P": "😜",
    "=d": "😜",
    "<3": "❤️"
}

In [39]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/joshua/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [40]:
def replace_emoticons(text):
    for emote_text, emote in emoticon_map.items():
        text = text.replace(emote_text, emote)
    return text

In [ ]:
def preprocess(text):
    text =  html.unescape(text)                                                     # decode html emojis
    text = replace_emoticons(text)                                                  # convert html emojis into actual emojis
    text = emoji.demojize(text, language="en")                                      # convert emojis into words
    text = text.lower()                                                             # lower case all words
    text = contractions.fix(text)                                                   # de-contract words
    text = re.sub(r'http\S+', '', text)                                             # remove links
    text = re.sub(r'@\S+', '', text)                                                # remove usernames
    text = re.sub(r"[,.`\"'-]",'', text)                                            # remove punctuation
    text = text.strip()                                                             # remove surrounding whitespace if any
    tokens = text.split()                                                           # tokenize
    tokens = [word for word in tokens if word not in stopwords.words('english')]    # remove stopwords
    return tokens

In [42]:
dt = load_dataset('stanfordnlp/sentiment140', trust_remote_code=True)

In [43]:
df_test = pd.DataFrame(dt['test'])
df_test['text'] = df_test['text'].apply(preprocess)

In [44]:
df_test

,text,date,user,sentiment,query
0,"[loooooooovvvvvveee, kindle2, dx, cool, 2, fan...",Mon May 11 03:17:40 UTC 2009,tpryan,4,kindle2
1,"[reading, kindle2, love, lee, childs, good, read]",Mon May 11 03:18:03 UTC 2009,vcu451,4,kindle2
2,"[ok, first, assesment, #kindle2, fucking, rock...",Mon May 11 03:18:54 UTC 2009,chadfu,4,kindle2
3,"[love, kindle2, mine, months, never, looked, b...",Mon May 11 03:19:04 UTC 2009,SIX15,4,kindle2
4,"[fair, enough, kindle2, think, perfect, :smili...",Mon May 11 03:21:41 UTC 2009,yamarama,4,kindle2
...,...,...,...,...,...
493,"[ask, programming:, latex, indesign?:, submitt...",Sun Jun 14 04:31:43 UTC 2009,proggit,2,latex
494,"[note, hate, word, hate, pages, hate, latex, s...",Sun Jun 14 04:32:17 UTC 2009,sam33r,0,latex
495,"[ahhh, back, *real*, text, editing, environmen...",Sun Jun 14 04:36:34 UTC 2009,iamtheonlyjosie,4,latex
496,"[trouble, iran, see, hmm, iran, iran, far, awa...",Sun Jun 14 21:36:07 UTC 2009,plutopup7,0,iran
